# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

# Widgets

In [0]:
dbutils.widgets.text("catalog", "project_smartdata")
dbutils.widgets.text("schema_source", "bronze")
dbutils.widgets.text("schema_sink", "silver")

# Constants

In [0]:
catalog         =  dbutils.widgets.get("catalog")
schema_source   =  dbutils.widgets.get("schema_source")
schema_sink     = dbutils.widgets.get("schema_sink")
table_category  = "category"
table_products  = "products"
table_stores    = "stores"
table_sales     = "sales"
table_warranty  = "warranty"


# Read and Cache catalogs

In [0]:
df_stores = spark.table(f"{catalog}.{schema_source}.{table_stores}")
df_products =  spark.table(f"{catalog}.{schema_source}.{table_products}");
df_stores.cache()
df_products.cache()


# KPIs

In [0]:
df_warranty = spark.table(f"{catalog}.{schema_source}.{table_warranty}")
df_warranty.display()